# SeeingKOR Project : Making Pygame
# 국제한국학과, 20181204, 김지안(Ji-an Kim)

## * 게임의 컨셉

1. 주제는 임진왜란 중 '이순신 VS 왜' 의 싸움이다.
2. 거북선은 조선으로 들어오는 안택선(왜구가 사용하던 배)을 막아야 한다.
3. 이 게임은 오락실의 비행기 슈팅 게임과 유사하다.
4. 전투 장면의 배경은 바다와 비슷해 보이도록 파란색으로 칠하였다.

## * 게임 방법

1. 거북선은 화면 맨 왼쪽에서 상하로 움직인다.
2. 적은 화면 맨 오른쪽에서 점점 왼쪽으로 움직인다.
3. 거북선은 불대포를 사용하여 안택선을 맞추어야 한다.
4. 게임 플레이어는 ↑ 또는 ↓ 키를 이용하여 거북선의 위치를 조정한다. 또한 스페이스바 키를 이용하여 불대포를 발사한다.
5. 거북선이 안택선을 못 맞추고 안택선이 거북선이 있는 곳을 넘어오면 Crash! 문구가 뜬다. (Game End)

## * 게임 코드 설명

### 1. 라이브러리 임포트

In [4]:
import pygame
import random
from time import sleep

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html


### 2. 게임에 사용되는 변수 정의

In [5]:
BLACK = (0,0,0)
RED = (255,0,0)
BLUE = (0,0,255)
pad_width = 900
pad_height = 550
fighter_width = 120
fighter_height = 80
enemy_width = 100
enemy_height = 100

우선 게임에서 사용될 수 있는 색(Black, Red, Blue)의 RGB값을 먼저 저장해 놓는다.
또한 pad_width 및 height는 게임화면의 크기를 정의한다.
fighter(거북선)의 가로, 세로크기는 120X80이고,
enemy(안택선)의 가로, 세로크기는 100X100이다.

### 3. 적을 맞춘 개수 계산하는 함수 (drawKills)

In [6]:
def drawKills(count):
    global gamepad
    font = pygame.font.SysFont(None, 30)
    text = font.render('Enemy Kills :' + str(count), True, (255,255,255))
    gamepad.blit(text,(0,0))

파이게임 화면 맨 왼쪽 상단에 'Enemy Kills: XX' 이 폰트 크기 30으로 써진다. 여기서 global은 전역변수를 사용하겠다는 뜻이다.

### 4. 'Crash!' 글씨를 넣는 함수 (writeMessage, crash)

In [7]:
def writeMessage(text):
    global gamepad
    textfont = pygame.font.Font('freesansbold.ttf', 80)
    text = textfont.render(text, True, RED)
    textpos = text.get_rect()
    textpos.center = (pad_width/2, pad_height/2)
    gamepad.blit(text,textpos)
    pygame.display.update()
    sleep(2)
    runGame()
    
def crash():
    global gamepad
    writeMessage('Crash!')

거북선이 적과 충돌했을 때 'Crash!'라는 메시지가 출력된다. 글씨 폰트는 'freesansbold.ttf', 크기는 80, 빨간색으로 화면 정중앙에 쓰인다. 또한 2초 정도 휴식 후 다시 게임이 재개된다. (sleep(2) -> rungame())

### 5. 객체를 화면에 나타내는 함수 (drawObject)

In [8]:
def drawObject(obj,x,y):
    global gamepad
    gamepad.blit(obj,(x,y))

이 게임에서는 거북선, 안택선, 대포 등 여러 가지 객체가 사용된다. 이 객체들을 하나의 함수로 묶어서 화면에 나타나도록 하였다. 나타내고자 하는 객체를 drawObject('객체 이름', (x좌표,y좌표)) 로 부르면 된다.

### 6. 게임 실행 메인 함수 (runGame)

In [ ]:
def runGame():
    global gamepad, clock, fighter, enemy, bullet
 
    isShot = False # 대포에 안택선이 맞았을 경우 True로 설정
    shotcount = 0
    enemypassed = 0
 
    # 대포 좌표를 위한 리스트
    bullet_xy = []
 
    # 거북선 초기 위치 (x,y)
    x = pad_width * 0.01 # 900 * 0.01, 화면 가로크기의 거의 맨 왼쪽
    y = pad_height * 0.5 # 550 * 0.5, 화면 세로크기의 중앙값
    y_change = 0
 
    # 안택선 초기위치 설정
    enemy_y = random.randrange(0,pad_height - enemy_height) # 안택선의 y 좌표는 랜덤값
    enemy_x = pad_width # 안택선의 x 좌표는 고정
    enemy_speed = 2 # 처음 속도는 2
 
    ongame = False
    while not ongame:
        for event in pygame.event.get():
            if event.type == pygame.QUIT: #마우스로 창을 닫는 이벤트
                doneFlag = True
 
            if event.type == pygame.KEYDOWN: # 키를 눌렀을 때
                if event.key == pygame.K_UP: # ↑ 화살표 키를 눌렀을 때
                    y_change -= 5 # 거북선이 위로 올라감
 
                elif  event.key == pygame.K_DOWN: # ↓ 화살표 키를 눌렀을 때
                    y_change += 5 # 거북선이 아래로 내려감
 
                # 스페이스바를 누르면 대포 발사
                elif  event.key == pygame.K_SPACE:
                    if len(bullet_xy) < 3:
                        bullet_x = x + fighter_width # 대포의 x 좌표
                        bullet_y = y + fighter_height/2 # 대포의 y 좌표
                        bullet_xy.append([bullet_x, bullet_y]) # x,y 좌표값을 나타냄
 
            if event.type == pygame.KEYUP: # 키를 누르지 않았을 때
                if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                    y_change = 0 # y의 변화값은 0
 
        gamepad.fill(BLUE) # 화면을 파란색으로 채우고 화면을 업데이트
         
        # 거북선 위치를 재조정
        y += y_change # 거북선의 위치는 기존 위치에서 y_change 값을 더함
        if y < 0:
            y = 0 # 거북선이 맨 위까지 올라가면 더 이상 올라가지 않음 (화면 밖을 벗어나지 못하도록)
        elif y > pad_height - fighter_height:
            y = pad_height - fighter_height
 
        # 거북선이 안택선과 충돌했는지 체크
        if x >= enemy_x:
                crash()
        drawObject(fighter,x,y) # 거북선을 게임 화면의 (x,y) 좌표에 그림
         
        # 거북선 대포 발사 화면에 그리기
        if len(bullet_xy) != 0:
            for i, bxy in enumerate(bullet_xy): # 총알 반복함
                bxy[0] += 10 # 총알의 x좌표를 +10함 (오른쪽으로 이동)
                bullet_xy[i][0] = bxy[0]
 
                # 대포가 안택선을 맞췄을 경우
                if bxy[0] > enemy_x:
                    if bxy[1] > enemy_y and bxy[1] < enemy_y + enemy_height:
                        bullet_xy.remove(bxy) # 대포를 제거
                        isShot = True # 거북선이 안택선을 맞춤
                        shotcount += 1
 
                if bxy[0] >= pad_width: # 대포가 화면 밖을 벗어나면 
                    try:
                        bullet_xy.remove(bxy)  # 대포를 제거
                    except:
                        pass
 
        if len(bullet_xy) != 0: # 대포가 최소 1개 이상이 되는 경우
            for bx,by in bullet_xy:
                drawObject(bullet,bx,by) # 대포를 화면에 나타냄
        
        drawKills(shotcount) # Enemy Kills: 00 나타냄
 
        # 안택선을 왼쪽으로 움직임
        enemy_x -= enemy_speed # 안택선은 주어진 속도만큼 x 좌표를 왼쪽으로 움직임
 
        if enemy_x < 0:
            enemy_x = 0 # 안택선이 맨 왼쪽까지 가면 그 좌표는 0이 됨
            enemy_y = random.randrange(0, pad_height - enemy_height) # y 좌표가 새롭게 설정됨
 
        # 적이 무기에 맞았는지 체크하고, 적이 무기에 맞았으면 속도를 올림
        if isShot:
            enemy_speed += 1
            if enemy_speed >= 10:
                enemy_speed = 10 # 속도의 최댓값은 10
 
            enemy_y = random.randrange(0, pad_height-enemy_height)
            enemy_x = pad_width
            isShot = False
        drawObject(enemy,enemy_x,enemy_y) # 안택선을 화면에 나타냄
         
        pygame.display.update() # 게임화면을 다시그림 
        clock.tick(60) # 게임화면의 초당 프레임수를 60으로 설정
    pygame.quit() # pygame을 종료

설명은 코드에 있는 주석(#)으로 대체한다.

### 7. 게임 초기화 함수(initGame)

In [4]:
def initGame():
    global gamepad,clock,fighter, enemy, bullet
    pygame.init()
    gamepad = pygame.display.set_mode((pad_width, pad_height)) #게임화면의 가로, 세로 크기를 설정
    
    pygame.display.set_caption('jian Game') #게임화면의 제목 지정
    
    # 게임에 사용될 이미지 불러오기
    fighter = pygame.image.load('koreaship.png') # 거북선 사진
    enemy = pygame.image.load('japanship.png') # 안택선 사진
    bullet = pygame.image.load('bullet.png') # 대포 사진
    
    clock = pygame.time.Clock() # 초당 프레임수를 설정할 수 있는 Clock객체 생성

### 8. 게임을 초기화한 후 실행

In [ ]:
initGame()
runGame()

게임을 초기화하고 실행시킨다.

## * 전체 코드

In [ ]:
import pygame
import random
from time import sleep

BLACK = (0,0,0)
RED = (255,0,0)
BLUE = (0,0,255)
pad_width = 900
pad_height = 550
fighter_width = 120
fighter_height = 80
enemy_width = 100
enemy_height = 100

def drawKills(count):
    global gamepad
    font = pygame.font.SysFont(None, 30)
    text = font.render('Enemy Kills :' + str(count), True, (255,255,255))
    gamepad.blit(text,(0,0))

def writeMessage(text):
    global gamepad
    textfont = pygame.font.Font('freesansbold.ttf', 80)
    text = textfont.render(text, True, RED)
    textpos = text.get_rect()
    textpos.center = (pad_width/2, pad_height/2)
    gamepad.blit(text,textpos)
    pygame.display.update()
    sleep(2)
    runGame()
    
def crash():
    global gamepad
    writeMessage('Crash!')

def drawObject(obj,x,y):
    global gamepad
    gamepad.blit(obj,(x,y))

def runGame():
    global gamepad, clock, fighter, enemy, bullet
 
    # 전투기 무기에 적이 맞았을 경우 True로 설정되는 플래그
    isShot = False
    shotcount = 0
    enemypassed = 0
 
    # 무기 좌표를 위한 리스트 자료
    bullet_xy = []
 
    # 거북선 초기 위치 (x,y)
    x = pad_width * 0.01 # 900 * 0.01, 화면 가로크기의 거의 맨 왼쪽
    y = pad_height * 0.5 # 550 * 0.5, 화면 세로크기의 중앙값
    y_change = 0
 
    # 안택선 초기위치 설정
    enemy_y = random.randrange(0,pad_height - enemy_height) # 안택선의 y 좌표는 랜덤값
    enemy_x = pad_width # 안택선의 x 좌표는 고정
    enemy_speed = 2 # 처음 속도는 2
 
    ongame = False
    while not ongame:
        for event in pygame.event.get():
            if event.type == pygame.QUIT: #마우스로 창을 닫는 이벤트
                doneFlag = True
 
            if event.type == pygame.KEYDOWN: # 키를 눌렀을 때
                if event.key == pygame.K_UP: # ↑ 화살표 키를 눌렀을 때
                    y_change -= 5 # 거북선이 위로 올라감
 
                elif  event.key == pygame.K_DOWN: # ↓ 화살표 키를 눌렀을 때
                    y_change += 5 # 거북선이 아래로 내려감
 
                # 스페이스바를 누르면 무기 발사
                elif  event.key == pygame.K_SPACE:
                    if len(bullet_xy) < 3:
                        bullet_x = x + fighter_width # 대포의 x 좌표
                        bullet_y = y + fighter_height/2 # 대포의 y 좌표
                        bullet_xy.append([bullet_x, bullet_y]) # x,y 좌표값을 나타냄
 
            if event.type == pygame.KEYUP: # 키를 누르지 않았을 때
                if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                    y_change = 0 # y의 변화값은 0
 
        gamepad.fill(BLUE) # 화면을 파란색으로 채우고 화면을 업데이트
         
        # 거북선 위치를 재조정
        y += y_change # 거북선의 위치는 기존 위치에서 y_change 값을 더함
        if y < 0:
            y = 0 # 거북선이 맨 위까지 올라가면 더 이상 올라가지 않음 (화면 밖을 벗어나지 못하도록)
        elif y > pad_height - fighter_height:
            y = pad_height - fighter_height
 
        # 게이머 전투기가 적과 충돌했는지 체크
        if x >= enemy_x:
                crash()
        drawObject(fighter,x,y) # 거북선을 게임 화면의 (x,y) 좌표에 그림
         
        # 거북선 총알 발사 화면에 그리기
        if len(bullet_xy) != 0:
            for i, bxy in enumerate(bullet_xy): # 총알 반복함
                bxy[0] += 10 # 총알의 x좌표를 +10함 (오른쪽으로 이동)
                bullet_xy[i][0] = bxy[0]
 
                # 총알이 안택선을 격추했을 경우
                if bxy[0] > enemy_x:
                    if bxy[1] > enemy_y and bxy[1] < enemy_y + enemy_height:
                        bullet_xy.remove(bxy) # 총알을 제거
                        isShot = True # 거북선이 안택선을 맞춤
                        shotcount += 1
 
                if bxy[0] >= pad_width: # 총알이 화면 밖을 벗어나면 
                    try:
                        bullet_xy.remove(bxy)  # 총알을 제거
                    except:
                        pass
 
        if len(bullet_xy) != 0: # 총알이 최소 1개 이상이 되는 경우
            for bx,by in bullet_xy:
                drawObject(bullet,bx,by) # 총알을 화면에 나타냄

        drawKills(shotcount)
 
        # 안택선을 왼쪽으로 움직임
        enemy_x -= enemy_speed # 안택선은 주어진 속도만큼 x 좌표를 왼쪽으로 움직임
 
        if enemy_x < 0:
            enemy_x = 0 # 안택선이 맨 왼쪽까지 가면 그 좌표는 0이 됨
            enemy_y = random.randrange(0, pad_height - enemy_height) # y 좌표가 새롭게 설정됨
 
        # 적이 무기에 맞았는지 체크하고, 적이 무기에 맞았으면 속도를 올림
        if isShot:
            enemy_speed += 1
            if enemy_speed >= 10:
                enemy_speed = 10 # 속도의 최댓값은 10
 
            enemy_y = random.randrange(0, pad_height-enemy_height)
            enemy_x = pad_width
            isShot = False
        drawObject(enemy,enemy_x,enemy_y) # 안택선을 화면에 나타냄
         
        pygame.display.update() # 게임화면을 다시그림 
        clock.tick(60) # 게임화면의 초당 프레임수를 60으로 설정
    pygame.quit() # pygame을 종료

def initGame():
    global gamepad,clock,fighter, enemy, bullet
    pygame.init()
    gamepad = pygame.display.set_mode((pad_width, pad_height)) #게임화면의 가로, 세로 크기를 설정
    
    pygame.display.set_caption('jian Game') #게임화면의 제목 지정
    
    # 게임에 사용될 이미지 불러오기
    fighter = pygame.image.load('koreaship.png')
    enemy = pygame.image.load('japanship.png')
    bullet = pygame.image.load('bullet.png')
    
    clock = pygame.time.Clock() # 초당 프레임수를 설정할 수 있는 Clock객체 생성

initGame()
runGame()

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html


## * 결론

지금까지 Seeing Korea 수업을 통해 pygame을 배우면서, 간단한 게임 하나를 만들더라도 생각보다 많은 노력이 들어간다는 것을 알게 되었다. 실제로 이번 프로젝트에 사용된 코드만으로도 150줄에 이른다. 간단한 픽셀 단위 게임도 이렇게 많은 코드가 들어가는데, 현재 게임시장에서 판매하는 게임들은 얼마나 많은 개발자의 노력이 들어가는지 감히 체험해볼 수 있었다. 국제한국학과 학생으로서, 지금까지 '게임'을 어떻게 하면 '한국학'과 연관지어 재밌게 표현할 수 있을 지에 대해 고민하고 실행해 보았다. 컴퓨터와 학문의 활용은 무궁무진하다. IT 세계를 살아가는 대학생으로써, '한국학'이란 학문을 다양한 분야와 함께 실천해 보는 좋은 기회였다고 생각한다.

## * 참고 자료

1. 책 <초보자를 위한 파이썬 200제>
2. ThePythonGameBook "http://thepythongamebook.com/start"
3. Program Arcade Games With Python And Pygame, Chapter 13
"http://programarcadegames.com/index.php?chapter=introduction_to_sprites&lang=ko#section_13"
4. Pygame Documentation "https://www.pygame.org/docs/"
